In [2]:
##################################################
# Eventstudy.py (PYTHON)
#     download CRSP & Compustat data.
#     by Xinyu
##################################################

import pandas as pd
import numpy as np
import wrds
import datetime as dt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", 50)
import os

from datetime import datetime
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Loading library list...
Done


In [91]:
direct = os.path.realpath('.')
filename = os.path.join(direct, 'Data','eventdate.csv')
eventdate = pd.read_csv(filename,parse_dates=['announcedate'])
varlist = ['announcedate','gvkey']
eventdate = eventdate[varlist]
eventdate.dropna(inplace=True)
eventdate.reset_index(drop=True,inplace=True)
eventdate.rename(columns={'announcedate':'datadate'},inplace=True)

In [92]:
#######################
# Dataset 2           #
# CCM Block           #
#######################
# Load CCM link table
varlist = "gvkey, lpermno, linkdt, linkenddt"
dataset = "crsp.ccmxpf_linktable"
ccm=conn.raw_sql(f"""
                    select {varlist}
                    from {dataset}
                    where substr(linktype,1,1)='L'
                    and (linkprim ='C' or linkprim='P')
                    """, date_cols=['linkdt','linkenddt'])
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today')+MonthEnd(0)).dt.date
ccm['gvkey']= ccm['gvkey'].astype(int)

In [93]:
# Merge comp with ccm to get a merged df
eventdate_m=pd.merge(eventdate,ccm,how='left',on=['gvkey'])
eventdate_m['yearend']=eventdate_m['datadate']+YearEnd(0)
eventdate_m['jdate']=eventdate_m['yearend']+MonthEnd(6)
# set link date bounds
eventdate_m=eventdate_m[(eventdate_m['jdate']>=eventdate_m['linkdt'])&(eventdate_m['jdate']<=eventdate_m['linkenddt'])]

In [94]:
varlist = ['datadate','lpermno']
eventdate_m = eventdate_m[varlist]
eventdate_m.rename(columns={'lpermno':'permno','datadate':'edate'},inplace=True)
eventdate_m.reset_index(drop=True,inplace=True)
eventdate_m['edate'] = eventdate_m['edate'].dt.strftime('%m/%d/%Y')
varlist = ['permno','edate']
eventdate_m = eventdate_m[varlist]
eventdate_m['permno'] = eventdate_m['permno'].astype(int)
evtstudy = eventdate_m.to_dict('records')
import json
with open('evtstudy-sample.json', 'w') as event:
    json.dump(evtstudy, event)